Cosumnes Model 
@author: Andrew

This notebook ran on 5/10/2021 and couldn't handle the increased number of layers

In [ ]:
# standard python utilities
import os
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
#import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:

# run installed version of flopy or add local path
try:
    import flopy
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference
    from flopy.utils import Raster
except:
    import flopy
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference
    from flopy.utils import Raster
from flopy.utils.gridgen import Gridgen
from flopy.utils import OptionBlock
import flopy.utils.binaryfile as bf


print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
# Transient -> might want to think about making SP1 steady
end_date = '2018-01-10'
# end_date = '2018-01-02'
strt_date = '2018-01-01'

dates = pd.date_range(strt_date, end_date)

# The number of periods is the number of dates 
nper = 1

# Each period has a length of one because the timestep is one day, have the 1st stress period be out of the date range
# need to have the transient packages start on the second stress period
perlen = np.ones(nper)
# Steady or transient periods
steady = np.ones(nper)
#steady[0] = 1 # first period is steady state, rest are transient
steady = steady.astype('bool').tolist()
# Reduce the number of timesteps to decrease run time
nstp = np.ones(nper)*np.append(np.ones(1),6*np.ones(nper-1))

In [ ]:
steady

In [ ]:
#Local Model Parameters
nrow=60
ncol=105
delr=200
delc=200
rotation=44.7

# The number of layers should be 1 for the Mehrten formation, 1 for the laguna plus the number of TPROGS layers,
# where the Laguna formation will be clipped by the TPROGS layers
# num_tprogs = 200
# nlay = 2 + num_tprogs
nlay = 200
tprog_thick = 0.5

# There is essentially no difference bewtween WGS84 and NAD83 for UTM Zone 10N
# proj4_str='EPSG:26910'
proj4_str='+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84 +units=m +no_defs '

In [ ]:
## Set up directory referencing
# Package data
git_dir = os.path.dirname(os.path.dirname(os.getcwd()))
gwfm_dir = '\\'.join(str.split(git_dir,'\\')[0:3])+ '/Box/research_cosumnes/GWFlowModel'
print(git_dir, gwfm_dir)

In [ ]:
from flopy.utils.geometry import Polygon, LineString, Point
# Original model domain, 44.7 deg angle
#m_domain = gpd.read_file(gwfm_dir+'\\GWModelDomain_UTM10N\\GWModelDomain_Rec_UTM10N.shp')
fig,ax=plt.subplots()
#m_domain.plot(ax=ax, color="None")
# New model domain 52.9 deg
#m_domain = gpd.read_file(gwfm_dir+'\\NewModelDomain\\GWModelDomain_52_9deg_UTM10N_WGS84.shp')
m_domain = gpd.read_file(gwfm_dir+'\\DIS_data\\Inset_grid_44_7\\MODFLOW_grid_final.shp')
m_domain.plot(ax=ax, color="None")

# Need to check this when changing model domains
xul, yul = list(m_domain.geometry.values[0].exterior.coords)[1]
list(m_domain.geometry.values[0].exterior.coords), xul, yul
# m_domain.geometry.values[0].exterior

According to Flopy GitHub "Technically you need to create both a SpatialReference object and a ModelGrid object, but in practice the code looks very similar and can easily be implemented in one line."
WGS84 Zone 10N has EPSG: 32610  
Lower left corner of model is   
Zone 10 N  
Easting: 661211.18 m E  
Northing: 4249696.50 m N  
angle is approximate 53 degrees  

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'

if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 

loadpth = loadpth +'/GWFlowModel/Cosumnes/Cosumnes_simple/'
model_ws = loadpth+'WEL_RIV_RCH_tprogs_'+str(nlay)+'layers'


In [ ]:
m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf2005.exe', 
                          version = 'mf2005', model_ws=model_ws)
# m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf2005.exe', 
#                           version = 'mf2005', model_ws='data')


In [ ]:
#lenuni = 1 is in ft, lenuni = 2 is in meters
# itmuni is time unit 5 = years, 4=days, 3 =hours, 2=minutes, 1=seconds
dis = flopy.modflow.ModflowDis(nrow=nrow, ncol=ncol, 
                               nlay=nlay, delr=delr, delc=delc,
                               model=m, lenuni = 2, itmuni = 4,
                               xul = xul, yul = yul,rotation=rotation, proj4_str=proj4_str,
                              nper = nper, perlen=perlen, nstp=nstp, steady = steady,
                              start_datetime = strt_date)

In [ ]:
# m.modelgrid.set_coord_info(xoff=xoff, yoff=yoff, proj4='EPSG:32610', angrot=angrot)
mg = m.modelgrid
# Write model grid to shapefile for later use
# mg.write_shapefile(gwfm_dir+'/DIS_data/grid/grid.shp', epsg = '32610')
# mg.write_shapefile(gwfm_dir+'/DIS_data/44_7_grid/44_7_grid.shp', epsg = '32610')
#mg.write_shapefile(gwfm_dir+'/DIS_data/Inset_grid_44_7/44_7_grid.shp', epsg = '3310')


In [ ]:
# Load model grid as geopandas object
grid_regional = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')
# grid_p = gpd.read_file(gwfm_dir+'/DIS_data/44_7_grid/44_7_grid.shp')
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/Inset_grid_44_7/44_7_grid.shp')

# print(gwfm_dir)

# Find Michigan Bar location
# mb_gpd = sensors[sensors.Sensor_id == "MI_Bar"]
# mb_grid = gpd.sjoin(mb_gpd, grid_p, how = 'left', op = 'intersects')

In [ ]:
grid_p_temp = grid_p.copy()
grid_p_temp.geometry = grid_p_temp.buffer(-99.9999)
# grid_regional.geometry = grid_regional.buffer(-100)

grid_ref = gpd.sjoin(grid_p_temp, grid_regional.to_crs('epsg:3310'), how = 'left',predicate = 'within',
                     lsuffix='inset', rsuffix = 'regional')
# grid_ref = pd.DataFrame(grid_ref.loc[:,['row_inset','column_inset','row_regional','column_regional']])
# reset geometry from tiny polygon to rectangle
grid_ref.geometry = grid_p.geometry

In [ ]:
grid_ref = grid_ref.rename({'row_inset':'row','column_inset':'column'},axis=1)
grid_ref.crs

In [ ]:
# Get vertexes of model domain
# ll = mg.get_coords(0, 0) #lower left
# lr = mg.get_coords(nrow*delr, 0) #lower right
# ur = mg.get_coords(nrow*delr, ncol*delc) #upper right
# ul = mg.get_coords(0, ncol*delc) #upper left
ll = mg.get_coords(0, 0) #lower left
lr = mg.get_coords(0, nrow*delr) #lower right
ur = mg.get_coords(ncol*delc, nrow*delr) #upper right
ul = mg.get_coords(ncol*delc, 0) #upper left
print(ll, lr, ur, ul)

# Shapefile of model bounds
from shapely.geometry import Polygon
vertices = np.stack(np.asarray((ll,lr, ur, ul)))
vertices
geoms = Polygon(vertices)

### Raster files can be loaded using the `Raster.load` method

In [ ]:
# Full size dem of northern sac valley
# raster_name = gwfm_dir+"/DEM_data/Cosumnes_DEM_3310.tif"
# rio10_utm = Raster.load(raster_name)

In [ ]:
# rio10_utm.plot()
# t0 = time.time()
# rio10_utm.crop(vertices, invert=False)
# crop_time = time.time() - t0
# rio10_utm.plot()


In [ ]:
# Nearest neighbor determines the nearest pixel and assumes its value
# linear is as it sounds, cubic is the smoothed version of linear essentially by using a cubic function
# the linear method takes a very, very long time - an hour plus??, just stick with nearest
# nearest takes 170.209 seconds - 220 seconds
# # the linear interpolation causes the program to crash
# t0 = time.time()
# dem_data = rio10_utm.resample_to_grid(m.modelgrid.xcellcenters,
#                                  m.modelgrid.ycellcenters,
#                                  band=rio10_utm.bands[0],
#                                  method="nearest")
# resample_time = time.time() - t0
# print("Resample time, nearest neighbor: {:.3f} sec".format(time.time() - t0))

In [ ]:
# np.savetxt(gwfm_dir+'\DIS_data\dem_44_7_200m_nearest.tsv', dem_data, delimiter = '\t')

# Based on Maribeth's grid aligned with Alisha's TPROGS model
# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_52_9_200m_nearest.tsv', delimiter = '\t')
dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_52_9_200m_linear.tsv', delimiter = '\t')
# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_44_7_200m_linear_missing_right_corner.tsv', delimiter = '\t')

# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_44_7_200m_nearest.tsv', delimiter = '\t')

#import seaborn as sns
#sns.heatmap(dem_data, cmap = 'viridis', vmin = 0)

In [ ]:
dem_inset = np.zeros((nrow,ncol))
dem_inset[grid_ref.row-1, grid_ref.column-1] = dem_data[grid_ref.row_regional-1, grid_ref.column_regional-1]
# renaming for use
dem_regional = np.copy(dem_data)
dem_data = np.copy(dem_inset)

In [ ]:
dem_data.crs

# Capture cross section of deeper geology

In [ ]:
# Pre-cretaceous metamorphic rocks - (variable thickness 200-500 ft thick)
# Ione formation (200 ft thick)
# Valley Springs formation (300 ft thick)
# Mehrten Formation (100 ft thick to 300 ft thick) (1-2 deg dip)
# Laguna Formation (less than 100 ft to between 200-300 ft thick) (less than 1 deg dip)
# upper formation (informed by well logs) (100 ft)
# ibound < 0 is constant head
# ibound = 0 is inactive cell
# ibound > 0 is active cell

## Need to adjust for change in model grid, based on Michigan Bar previously, maybe also look at effect of model domain angle vs cross section angle

In [ ]:

# The stream gage at michigan bar is now 13 columns in from the boundary
# mehrtenbound

# Cross section E appears to have an angle of 0 compared to the model domain,

In [ ]:
# # columns are xtop_miles, ytop_ft_amsl, xbot_miles, ytop_ft_amsl
# # XS upper bound should be at Michigan bar which is between Jackson road and Sacramento-Amador county line split
# # Mile 36 is approximately where Michigan bar aligns with the cross section
MB_XS_mile = 36
mehrtenbound = pd.read_csv(gwfm_dir+'/DIS_data/Mehrten_boundary_x_y.csv', parse_dates = False, 
                index_col = False, sep = ',', header = 'infer')
# Convert miles to feet and sets x value based on location of Michigan bar
# 0 is michigan bar and each major change in geologic dip is based on distance from Michigan Bar
mehrtenbound.xtop_miles = -5280*(MB_XS_mile - mehrtenbound.xtop_miles)
mehrtenbound.xbot_miles = -5280*(MB_XS_mile - mehrtenbound.xbot_miles)
# No flod boundary based on the original coordinates of the bottom of the Mehrten formation
mehrtenbound.noflow_x_miles = -5280*(MB_XS_mile - mehrtenbound.noflow_x_miles)

# East of mile 32 the entire vertical cross section, including up to the near entire surface
# is composed of old geologic formations that are not water bearing
volcanic_bound = (MB_XS_mile - 32)*-5280
# noflow_ind = int((1-(volcanic_bound/sumx))*ncol)

# Plot the x and y values
fig, ax = plt.subplots(1, 1)
mehrtenbound.plot(x = 'xtop_miles', y = 'ytop_ft_amsl', ax = ax)
mehrtenbound.plot(x = 'xbot_miles', y = 'ybot_ft_amsl', ax = ax)
plt.plot(-100*3.28*np.arange(0,len(dis.top[40,:])), np.flip(3.28*dis.top[40,:]))
# print(mehrtenbound)

In [ ]:

def xy_to_botm(xbound_ft, ybound_ft, nspace,ntransversespace):
    laybotm = np.zeros((ntransversespace, nspace))
    # Nspace will be either nrow or ncol depending model direction
    # ntransversespace is the opposite of nspace (ie nrow if nspace is ncol)
    # Calculate the distance between each major change in dip
    dx = np.diff(xbound_ft)
    # Scale by the total distance across the coordinates to get percentages
    sumx = np.sum(dx)
    dx /= sumx
    # Multiply the number of columns by the percent of columns in each section of constant dip
    dx *= nspace
    # Round the number of columns to allow proper use for indexing
    nx = np.round(dx).astype(int)
    # Fix any discrepancy in number of columns due to issues with rouding the percentages of columns
    # Add a column to the last set of columns because there is already uncertainty at the deeper end
    while(np.sum(nx)-nspace !=0):
        if np.sum(nx)-nspace <0:
            nx[-1] += 1
        elif np.sum(nx)-nspace >0:
            nx[-1] -= 1
    sum(nx)

    # Now split the coordinates into downsized coordinates in between each major change in dip
    k = 0
    for i in np.arange(0,len(nx)):
        for j in np.arange(0,ntransversespace):
            laybotm[j, k:k+nx[i]] = np.arange(ybound_ft[i],ybound_ft[i+1], -(ybound_ft[i]-ybound_ft[i+1])/nx[i])
        k += nx[i]
    return(laybotm)

In [ ]:
# # X (east -west) and y (up-down vertical) of major dip changes for Mehrten Formation top boundary

# mehrten_top = xy_to_botm(mehrtenbound.xtop_miles,mehrtenbound.ytop_ft_amsl, ncol, nrow)
# # X (east -west) and y (up-down vertical) of major dip changes for Mehrten Formation bottom boundary
# # drop na is because there are less values to mark changes in the bottom than top boundary
# mehrten_bottom = xy_to_botm(mehrtenbound.xbot_miles.dropna(),mehrtenbound.ybot_ft_amsl.dropna(), ncol, nrow)

# # Original x,y data for Mehrten bottom boundary to represent the noflow bounds
# no_flow_bound = xy_to_botm(mehrtenbound.noflow_x_miles.dropna(), mehrtenbound.noflow_y_ft_amsl.dropna(),ncol,nrow)

In [ ]:
# botm = np.zeros((nlay, nrow, ncol))
# # Convert units from ft to meters and flip to match direction
# botm[-2,:,:] = np.flip(mehrten_top/3.28)
# botm[-1,:,:] = np.flip(mehrten_bottom/3.28)
# no_flow_bound = np.flip(no_flow_bound/3.28)
# dis.botm = botm
# # dis.plot()

## Adjustment to bottom boundary to ensure sufficient top layer thickness for the TPROGS model
Although the bottom boundaries are being artifically lowered to allow for sufficient layer thickness, this will be corrected when ibound is implemented based on where the actual bottom boundary is and where there is high elevations based on likelihood to be volcanics geology.

In [ ]:
# The TPROGS model is 100m thick with some of it above the land surface
# to be safe, there should be at least 50 meters below ground surface to the bottom boundary

# Create TPROGS layers from bottom up
# tprog_thick = 100/num_tprogs
# botm[-3,:,:] = -80
# for i in np.arange(-4,-3-num_tprogs,-1):
#     botm[i,:,:] = botm[i+1,:,:] + tprog_thick
    
    
# Create TPROGS layers from top down
# tprog_thick = 100/num_tprogs
# botm[0,:,:] = dem_data - tprog_thick
# botm[0,:,:] = -10
# for i in np.arange(1,num_tprogs):
#     botm[i,:,:] = botm[i-1,:,:] -tprog_thick
    
# # Thickness to give to bottom layers below the TPROGS layers just to provide adequate spacing,
# # this will be corrected by changing the geology in the layers above to account for what is actually in
# # the Mehrten and what is in the Laguna formations, thickness of 5 also prevents any messy overlap
# thickness_to_skip =10
# # # Find where top boundary of Mehrten Formation rises within 10 meters of the top layer (10m for sufficient layer thickness)
# bot3ind = np.min(np.where(botm[-2,:,:]>botm[-3,:,:]- thickness_to_skip)[1])

# # # Where the top boundary of Mehrten was within 10 meters of the top layer 
# # # set it equal to top layer elevation minus 10 for sufficient layer thickness
# botm[-2,:,bot3ind:] = botm[-3,0,bot3ind]- thickness_to_skip
# # # Repeat steps above for bottom of Mehrten formation with the top of the Mehrten formation
# bot3ind = np.min(np.where(botm[-1,0,:]>botm[-2,0,:]- thickness_to_skip))
# botm[-1,:,bot3ind:] = botm[-2,0,bot3ind]-thickness_to_skip

In [ ]:
# rectangular set up of botm from -50 to 50
botm = np.zeros((nlay, nrow, ncol))
for n, elev in enumerate(np.arange(-50,50,0.5)):
    botm[len(botm)-1-n,:,:] = elev


In [ ]:
# Set the elevation of the top layer based on the DEM
# m.dis.top = dem_data
m.dis.top = 50
# Bottom of model based on geology
m.dis.botm = botm
# chk = dis.check()
# chk.summary_array

In [ ]:
# dis.write_file()

In [ ]:

fig,ax = plt.subplots(figsize = (12,12))
plt.plot(dem_data[8,:])

for i in np.arange(0,nlay):
    plt.plot(botm[i,8,:])

## Import soil data for Lake Package, UZF Package, SFR Package hydraulic parameters

In [ ]:
# mb_name = gwfm_dir+"/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp"

# mb = gpd.read_file(mb_name)
# mb = mb.to_crs('epsg:32610')

In [ ]:
uzf_path = gwfm_dir+'\\UZF_data'
soil_path = uzf_path+'\\wss_gsmsoil_CA'
# # Read in the soil map spatial data
# soil_gpd = gpd.read_file(uzf_path+'\\wss_gsmsoil_CA\\spatial\\gsmsoilmu_a_ca.shp')
# soil_gpd = soil_gpd.to_crs('EPSG:32610')
# # soil_gpd.plot()

### Write grid_uzf to shapefile to avoid having to repeat analysis

In [ ]:
# grid_uzf.to_file(uzf_path+'/final_grid_uzf/griduzf.shp')
# grid_uzf = gpd.read_file(uzf_path+'/final_grid_uzf/griduzf.shp')

In [ ]:
# def fill_uzf(uzfvalues, grid_uzf):
#     # convert geopandas object to regular np array for soil data
#     temp = np.zeros((nrow,ncol))
#     temp[(grid_uzf.row.values-1).astype(int),(grid_uzf.column.values-1).astype(int)] = uzfvalues
#     return(temp)

In [ ]:
soilKs_array = np.loadtxt(uzf_path+'/final_soilKs.tsv', delimiter = '\t')
soiln_array = np.loadtxt(uzf_path+'/final_soiln.tsv', delimiter = '\t')
soileps_array = np.loadtxt(uzf_path+'/final_soileps.tsv', delimiter = '\t')

# soilKs_array = fill_uzf(grid_uzf.Ksat_Rep, grid_uzf)
# soiln_array = fill_uzf(grid_uzf.Porosity_R, grid_uzf)
# soileps_array = fill_uzf(grid_uzf.EPS, grid_uzf)

# np.savetxt(uzf_path+'/final_soilKs.tsv', soilKs_array, delimiter = '\t')
# np.savetxt(uzf_path+'/final_soiln.tsv', soiln_array, delimiter = '\t')
# np.savetxt(uzf_path+'/final_soileps.tsv', soileps_array, delimiter = '\t')

# RIV

In [ ]:
sfr_dir = gwfm_dir+'/SFR_data/'

In [ ]:
# grid_sfr.to_file(sfr_dir+'/final_grid_sfr/grid_sfr.shp')
grid_sfr = gpd.read_file(sfr_dir+'/final_grid_sfr/grid_sfr.shp')
grid_sfr = grid_sfr.rename(columns = {'row':'row_regional','column':'column_regional'})
grid_sfr= grid_sfr.set_index(['row_regional','column_regional'])
grid_sfr = grid_sfr.join(grid_ref.set_index(['row_regional','column_regional']), how='inner',rsuffix = 'grid' )


In [ ]:
# alternate method for row referncing
# grid_ref.set_index(['row_regional','column_regional']).loc[grid_sfr]
# grid_sfr.loc[grid_ref.loc[:,['row_regional','column_regional']].values.tolist()]

# join local and regional grid for subsetting river package
# grid_sfr = grid_sfr.join(grid_ref.set_index(['row_regional','column_regional']), how='inner',rsuffix = 'grid' )


In [ ]:
# XS8pt = pd.read_csv(sfr_dir+'8pointXS.csv')
# XSlocs = gpd.read_file(sfr_dir+'8pointXS_locs\\8pointXS_locs.shp')
# XSlocs.crs = 32610

# XSg  = gpd.sjoin(grid_sfr, XSlocs, how = "inner", op= "contains", lsuffix = 'sfr',rsuffix = 'xs')
# # Append the grid_breach location to the list of cross sections to split the segment
# XSg = XSg.append(grid_breach).sort_values('reach')
# # Copy the XS site name from the previous last site to the breach site to keep same XS
# XSg.Site.iloc[-1] = XSg.Site.iloc[-2]
# len(XSg), len(XS8pt.loc[0,:])/2

### Read in 8 pt XS, revised by simplifying from Constantine 2001

In [ ]:
# xs_sfr = grid_sfr.set_index('reach')
# # set all reaches to start as segment 1 which will be changed iteratively based on the number of cross-sections
# xs_sfr['iseg'] = 1
# # add a column reach_new that will be changed iteratively as the segment number is changed
# xs_sfr['reach_new'] = xs_sfr.index
# # xs_sfr

## Define reach data based on ISFROPT

In [ ]:
## Given the reach number of each XS, the 718 reaches will be broken down into each segment
## create a new reach column based on XS reach number and 

# segcount = 2
# for i in np.arange(0,len(XSg)):
#     temp_reach = XSg.reach.values[i]
#     rchnum = xs_sfr.index[-1] - temp_reach+1
#     xs_sfr.reach_new.loc[temp_reach:] = np.linspace(1,rchnum, rchnum)
#     xs_sfr.iseg.loc[temp_reach:] = segcount
#     segcount +=1

In [ ]:
# xs_sfr.reach_new = xs_sfr.reach_new.astype(int)


In [ ]:
grid_sfr.row_regional = (grid_sfr.row.values-1).astype(int)
grid_sfr.column_regional = (grid_sfr.column.values-1).astype(int)

In [ ]:
# Determine which layer the streamcell is in
# since the if statement only checks whether the first layer is greater than the streambed elevation, 
# otherwise it would be less than and zero (most should be in layer 0)
sfr_lay = np.zeros(len(grid_sfr))

for i in np.arange(0,nlay-1):
    # pull out elevation of layer bottom
    lay_elev = botm[i, grid_sfr.row.values, grid_sfr.column.values]
    for j in np.arange(0,len(grid_sfr)):
        # want to compare if streambed is lower than the layer bottom
        # 1 will be subtracted from each z value to make sure it is lower than the model top in the upper reaches
        if lay_elev[j] < (grid_sfr.z.values-1)[j]:
            sfr_lay[j] = i 
        


In [ ]:
# rate is meters per day
vertk = soilKs_array[grid_sfr.row, grid_sfr.column]
# vertk = 1

strbd_thick = 1.5 #estimate of thickness meters

In [ ]:
# find average stream width
# xswidth = XS8pt.iloc[:,np.arange(0,XS8pt.shape[1],2)]
# widths = xswidth.iloc[-1,:] - xswidth.iloc[0,:]
# avgwidth = np.mean(xswidth.iloc[-1,:] - xswidth.iloc[0,:])

### COnductance for RIVer

In [ ]:
avgwidth = 50 # expected width
# lay, row, col, stage, cond, rbot
grid_sfr
cond = grid_sfr.length_m*avgwidth*vertk/strbd_thick
# riv_spd = np.append(sfr_lay, [grid_sfr.row.values,grid_sfr.column.values,
#           grid_sfr.z.values, cond,grid_sfr.z.values-1.5],axis=1)

In [ ]:
# array of all data
riv_spd = np.transpose(np.vstack([sfr_lay,grid_sfr.row.values,grid_sfr.column.values,
          grid_sfr.z.values, cond,grid_sfr.z.values-1.5])) # 1.5 to make sure below model top

In [ ]:
riv_dict = {0:riv_spd}
riv_dict

In [ ]:
riv = flopy.modflow.ModflowRiv(model = m, stress_period_data=riv_dict)

## Complex ibound

### Define no flow cells based on elevation, informed by DWR cross sections and geologic maps of volcanic geology fingers leaving the mountains
In general, the location of Michigan Bar is near the boundary where there is total volcanics to majority alluvium. However there is a major finger North and South of the Cosumnes River of andesititc conglomerate, sandstone, breccia

In [ ]:
# Simplified ibound, only no flow cell if it is below the bottom of the Mehrten Formation
# Specify no flow boundary based on rough approx of geology (upper basin volcanics)
ibound = np.ones([nlay, nrow,ncol])
strt = np.ones((nlay, nrow, ncol), dtype = np.float32)
# The model should start in hydraulic connection
# strt[:,:,:] = np.copy(dem_data)
strt[:,:,:] = m.dis.top.array
# strt
# cutoff_elev = 56
# ibound = ibound*(dem_data<cutoff_elev)

# plt.imshow(ibound[0,:,:])

In [ ]:
# Basic package, BAS

# ibound < 0 is constant head
# ibound = 0 is inactive cell
# ibound > 0 is active cell
# strt is array of starting heads
bas = flopy.modflow.ModflowBas(model = m, ibound=ibound, strt = strt)
bas.check()
# bas.write_file()

### Create a line bounding the noflow region to set the specified head boundary

In [ ]:
# import pprint
# from rasterio.features import shapes, rasterize

# # The function shapes from rasterio requires uint8 format
# ibound_line = ibound.astype(rasterio.uint8)
# out = shapes(ibound_line,connectivity = 8)
# alldata = list(out)

# # maxl = 0
# maxl = np.zeros(len(alldata))
# for i in np.arange(0,len(alldata)):
#     maxl[i] = len(alldata[i][0].get('coordinates')[0])
# #     if len(alldata[i][0].get('coordinates')[0])>maxl:
# #         maxl = len(alldata[i][0].get('coordinates')[0])
# #         ind = i
# # select the two longest linestring indexes (1st will be chunk down of divide (lower elevation) 2nd will chunk above (high elev))
# maxl1, maxl2 = np.where(maxl>np.mean(maxl))[0]
# print(maxl[maxl>np.mean(maxl)])

In [ ]:
# temp = alldata[maxl2][0].get('coordinates')[0]
# tl = LineString(temp)
# tl

In [ ]:
# from shapely.ops import LineString, linemerge, polygonize, unary_union
# tl = LineString(temp)

# # Get the constant head or general head boundary after the no flow cells
# linerast = rasterio.features.rasterize([tl], out_shape = np.array((nrow,ncol)))
# # remove far east bound line
# linerast[:,ncol-1] = 0
# fix_bound = np.min(np.argwhere(linerast[0,:]==1))
# linerast[0,:] = 0
# linerast[0,fix_bound]
# np.shape(linerast)

# # ibound[0,linerast==1] = -1

In [ ]:
# from shapely.ops import LineString, linemerge, polygonize, unary_union
# tl = LineString(temp)
# tu = unary_union(tl)
# poly = list(polygonize(tu))
# # Set the polygon/raster for the top layer, no buffer needed
# poly0 = poly[0].buffer(distance = 0)
# polyrast0 = rasterio.features.rasterize([poly0], out_shape = np.array((nrow,ncol)))
# # Set the polygon/raster for the top layer, slight buffer needed to expand geologic formation outward with depth as 
# # naturally occurs
# poly1 = poly[0].buffer(distance = 13)
# polyrast1 = rasterio.features.rasterize([poly1], out_shape = np.array((nrow,ncol)))
# # Set the polygon/raster for the bottom layer, largest buffer needed
# poly2 = poly[0].buffer(distance = 17)
# polyrast2 = rasterio.features.rasterize([poly2], out_shape = np.array((nrow,ncol)))

# ibound = np.ones([nlay, nrow,ncol])
# # Need to decide whether all layers or just the top layer are affected by ibound from elevation
# # it is better to define the top layer with a simple dem>elevation check than the rasterize functins that isn't perfect
# # ibound[0,polyrast0==1] = 0
# # Need to decide whether all layers or just the top layer are affected by ibound from elevation
# ibound[-2,polyrast1==1] = 0
# # Need to decide whether all layers or just the top layer are affected by ibound from elevation
# ibound[-1,polyrast2==1] = 0

# # The bottom boundary has a dip of 1-2 degrees which is essentially a slope of 0.015 based on given cross section data
# # The layer thickness for TPROGS
# laythk = tprog_thick
# # It appeared shapely buffer is on the scale of kilometers
# run = (laythk/0.015)/1000
# run_const = run
# for i in np.arange(1,nlay-2):
#     # error saying poly[i] is not subscriptable
#     polyi = poly[0].buffer(distance = run)
#     polyrast = rasterio.features.rasterize([polyi], out_shape = np.array((nrow,ncol)))
#     # Need to decide whether all layers or just the top layer are affected by ibound from elevation
#     ibound[i,polyrast==1] = 0
#     run += run_const

In [ ]:
# # wherever the constant head/specified head bound is the cells need to be active
# ibound[0,dem_data>cutoff_elev] = 0
# ibound[0,linerast==1] = 1
# plt.imshow(ibound[0,:,:])
# plt.colorbar()

In [ ]:
# # copy the ibound array to alter the geology array to set these cells as low permeability formations
# # either marine or volcanic based
# deep_geology = ibound[:,:,:]

# # reset ibound to all active cells to reduce non-linearity
# # still need to take account of no flow cells for lake package
# ibound = np.ones([nlay, nrow,ncol])

### Remove no flow cells in the first layer where there are stream cells

In [ ]:
# # Get indices of SFR cells
# sfr_x = grid_sfr.row.values.astype(int)
# sfr_y = grid_sfr.column.values.astype(int)
# sfr_z = sfr_lay.astype(int)
# # Check where SFR cells overlap the no flow cell array 
# overlap = ibound[sfr_z, sfr_x, sfr_y]==0
# # Convert the cells that overlap from inactive to active cells
# ibound[sfr_z[overlap],sfr_x[overlap], sfr_y[overlap]] = 1
# # Check where SFR cells overlap the constant head cell array 
# overlap = ibound[sfr_z, sfr_x, sfr_y]==-1
# # Convert the cells that overlap from inactive to active cells
# ibound[sfr_z[overlap],sfr_x[overlap], sfr_y[overlap]] = 1

In [ ]:
# Basic package, BAS

# ibound < 0 is constant head
# ibound = 0 is inactive cell
# ibound > 0 is active cell
# strt is array of starting heads
# bas = flopy.modflow.ModflowBas(model = m, ibound=ibound, strt = strt)

In [ ]:
# bas.check()


"When subsurface recharge (MBR2) is negligible,
stream runoff at the mountain front (runoff measured at
point B in Figure 1, or RO) may be considered the total
contribution to MFR [Anderholm, 2000]." (Wilson and Guan 2004)

## General head boundary representing Delta/Sea Level

In [ ]:
ghb = flopy.modflow.ModflowGhb(model = m)

## Northwest and Southeast GHB boundaries based on historical WSEL

In [ ]:
# raster cropping will be done in outside script so the only part read in will be the final array
ghb_dir = gwfm_dir+'/GHB_data'

In [ ]:
strtyear = 2013
endyear = 2019
kriged_fall = np.zeros((int(endyear-strtyear),nrow,ncol))
kriged_spring = np.zeros((int(endyear-strtyear),nrow,ncol))

kriged_NW = np.zeros((int(endyear-strtyear)*2,ncol))
kriged_SE = np.zeros((int(endyear-strtyear)*2,ncol))
# keep track of which place in array matches to year
year_to_int = np.zeros((endyear-strtyear,2))
# array referencing
x = grid_ref.loc[:,['row_regional']].values.astype(int)-1
y= grid_ref.loc[:,['column_regional']].values.astype(int)-1
x_in = grid_ref.loc[:,['row']].values.astype(int)-1
y_in= grid_ref.loc[:,['column']].values.astype(int)-1

t=0
for year in np.arange(strtyear,endyear):
    
    # load and place spring kriged data in np array, load spring first
    filename = glob.glob(ghb_dir+'/final_WSEL_arrays/spring'+str(year)+'_kriged_WSEL.tsv')[0]
    df_grid = np.loadtxt(filename)
    kriged_spring[t,x_in,y_in] = df_grid[x,y]
    
    # load and place fall kriged data in np array
    filename = glob.glob(ghb_dir+'/final_WSEL_arrays/fall'+str(year)+'_kriged_WSEL.tsv')[0]
    df_grid = np.loadtxt(filename)
    kriged_fall[t,x_in,y_in] = df_grid[x,y]

    
    # save NW  dataframe
    kriged_NW[t] = kriged_spring[t,0,:]
    kriged_NW[2*t] = kriged_fall[t,0,:]
    # save SE data frame
    kriged_SE[t] = kriged_spring[t,nrow-1,:]
    kriged_SE[2*t] = kriged_fall[t,nrow-1,:]
    
    year_to_int[t,0] = t
    year_to_int[t,1] = year
    t+=1
    

In [ ]:
y_num=0

nwhead_fall = kriged_fall[:,0,:]
sehead_fall = kriged_fall[:,-1,:]


# Set kriged water table elevations that are above land surface to land surface
nwhead_fall = np.where(nwhead_fall>dem_data[0,:], dem_data[0,:], nwhead_fall)
sehead_fall = np.where(sehead_fall>dem_data[-1,:], dem_data[-1,:], sehead_fall)

kriged_NW = np.where(kriged_NW>dem_data[0,:], dem_data[0,:], kriged_NW)
kriged_SE = np.where(kriged_SE>dem_data[-1,:], dem_data[-1,:], kriged_SE)

# calculate the average depth to water table for the spring and from 2013-2018 for the northwest and southeast boundary
avg_nw = np.nanmean(kriged_NW,axis=0)
avg_se = np.nanmean(kriged_SE,axis=0)

In [ ]:
# steady state GHB layering
nw_lay = np.zeros(ncol)
se_lay = np.zeros(ncol)
for k in np.arange(0,nlay-1):
        # pull out elevation of layer bottom
        lay_elevnw = botm[k, 0, :]
        lay_elevse = botm[k, -1, :]
        for j in np.arange(0,ncol):
            if lay_elevnw[j] > avg_nw[j]:
                nw_lay[j] = k+1
            if lay_elevse[j] > avg_se[j]:
                se_lay[j] = k+1
# correct from float to integer for cell referencing
nw_lay = nw_lay.astype(int)
se_lay = se_lay.astype(int)

## Read in TPROGS data

In [ ]:
# "C:\Users\ajcalder\Box\research_cosumnes\Large_TPROGS_run\TPROGS_realizations\tsim_Cosumnes_Full_Model.asc1"
# create tprogs directory reference to 100 large tprogs runs ascii files
# tprogs_dir = os.path.dirname(gwfm_dir)+'/Large_TPROGS_run/Archive/TPROGS_realizations/'
# # get all file names
# tprogs_files = glob.glob(tprogs_dir+'*')


In [ ]:
# "C:\TProGS10\tsim"
mf_tprogs_dir = 'C:/TProGS10/tsim/Cosumnes_Alisha/'
# mf_tprogs_dir = gwfm_dir+'/UPW_data/tprogs_final/'
tprogs_files = glob.glob(mf_tprogs_dir+'*')


In [ ]:
def tprogs_cut_elev(tprogs_line):
    tprogs_arr = np.reshape(tprogs_line, (105, 60 ,200))
    tprogs_elev = np.copy(tprogs_arr)
    # the bottom layer of the tprogs model is at -50 m amsl and the top layer is 50 m amsl
    t = 0
    for k in np.arange(-50,50,0.5):
        tprogs_elev[t,dem_data<k]= np.NaN
        t+=1
    masked_tprogs = ma.masked_invalid(tprogs_elev)
    return(masked_tprogs)

In [ ]:
import numpy.ma as ma
def tprogs_cut_saturated(tprogs,kriged):
    tprogs_unsat = np.copy(tprogs)
    # the bottom layer of the tprogs model is at -80 m amsl and the top layer is 80 m amsl
    # set any tprogs cells below the average fall water table depth as np.nan
    t = 0
    for k in np.arange(-50,50,0.5):
        tprogs_unsat[t,kriged>k]= np.NaN
        t+=1
    masked_tprogs = ma.masked_invalid(tprogs_unsat)
    return(masked_tprogs)

In [ ]:
def int_to_param(tprogs):
    tprogs[tprogs<0] *= -1
    tprogs = tprogs.astype(float)
    # flip tprogs model along z axis to match modflow definition of 0 as top (TPROGS says 0 is bottom)
    tprogs = np.flip(tprogs,axis=0)
    tprogs_K = np.copy(tprogs)
    tprogs_Sy = np.copy(tprogs)
    tprogs_Ss = np.copy(tprogs)
    # hydraulic parameters from fleckenstein 2006
    # I-IV gravel, sand, muddy sand, mud
    # K in m/s, Sy, Ss
    params = np.asarray([[4e-3,1.5e-3,2.5e-4,6.5e-6],
                         [0.25,0.2,0.15,0.1],
                         [2.0e-5,8e-5,2e-4,5e-4]])
    # convert from m/s to m/d
    params[0] *= 86400    
    for n in np.arange(1,5):
        tprogs_K[tprogs==n]= params[0,n-1]
    for n in np.arange(1,5):
        tprogs_Sy[tprogs==n]= params[1,n-1]
    for n in np.arange(1,5):
        tprogs_Ss[tprogs==n]= params[2,n-1]
            
    return(tprogs_K,tprogs_Sy,tprogs_Ss)


In [ ]:
t=0
tprogs_line = np.loadtxt(tprogs_files[t], skiprows = 1)
masked_tprogs = np.reshape(tprogs_line, (200, 60, 105 ))
# masked_tprogs=tprogs_cut_elev(tprogs_line)
K, Sy, Ss= int_to_param(masked_tprogs)


In [ ]:
plt.imshow(K[0,:,:],aspect = 20)
# plt.imshow(K[:,:,0])

In [ ]:
# take the mean water surface elevations for fall and spring
kriged_fall_avg = np.nanmean(kriged_fall,axis=0)
kriged_spring_avg = np.nanmean(kriged_spring,axis=0)
# take average between fall and spring water surface elevations
kriged_fall_spring_avg = (kriged_fall_avg+kriged_spring_avg)/2

In [ ]:
# from scipy.stats import hmean
# unsat_hk = np.mean(tprogs_cut_saturated(K,kriged_fall_spring_avg),axis=0)
# unsat_vka = hmean(tprogs_cut_saturated(K,kriged_fall_spring_avg))
# unsat_sy = np.mean(tprogs_cut_saturated(Sy,kriged_fall_spring_avg),axis=0)
# unsat_ss = np.mean(tprogs_cut_saturated(Ss,kriged_fall_spring_avg),axis=0)

# # set any zero values to the average of the domain, should be added to the function above
# # arithmetic mean is used because we are filling data and not calculating vertical averages
# unsat_hk[unsat_hk.data==0] = np.mean(unsat_hk)
# unsat_vka[unsat_vka.data==0] = np.mean(unsat_vka)
# unsat_sy[unsat_sy.data==0] = np.mean(unsat_sy)
# unsat_ss[unsat_ss.data==0] = np.mean(unsat_ss)

In [ ]:
# the first layer starts at -10 m which corresponds to 20 layers below 0
# 0m AMSL is 160 layers above the bottom of tprogs which ends up giving 160-20 is layer 140 or 139 for 0 based

## GEL package

In [ ]:
hk = np.zeros(botm.shape)
vka = np.zeros(botm.shape)
sy = np.zeros(botm.shape)
ss = np.zeros(botm.shape)

In [ ]:
hk = np.flip(K,axis=0)
vka = np.flip(K,axis=0)
sy = np.flip(Sy,axis=0)
ss = np.flip(Ss,axis=0)

In [ ]:
# take of 2 for the bottom layers and 1 for the unsat zone layer up top
# for tprogs arrays 0 is the bottom of the model, so flipping on z will fix
# hk[1:-2,:,:] = np.flip(K[-hk.shape[0]+1:-2,:,:],axis=0)
# sy[1:-2,:,:] = np.flip(Sy[-sy.shape[0]+1:-2,:,:],axis=0)
# ss[1:-2,:,:] = np.flip(Ss[-ss.shape[0]+1:-2,:,:],axis=0)


In [ ]:
# set parameters based on upscaled unsaturated zone
# def unsat_hp(upw_array, hp_value):
#     temp = np.copy(upw_array)
#     temp[0,:,:] = hp_value
#     return(temp)
# # convert hydr. cond. from m/s to m/d
# hk = unsat_hp(hk, unsat_hk)
# vka = unsat_hp(vka, unsat_vka)
# sy = unsat_hp(sy, unsat_sy)
# ss = unsat_hp(ss,unsat_ss)

The tuff breccia is very dense, hard and low water yielding. It is supposedly responsible for the many "haystack" hills in the eastern part of the county

DWR report has a few final well pumping rates, drawdowns and specific capacities but limited.

Fleckenstein et al. 2006 found the Mehrten had
Kh = 1 to 1.8 x10^-5 m/s
Kv = 1 to 1.8 x10^-7 m/s
Sy = 0.15 to 0.2
Ss = 1e-4 to 1e-3 m^-1

In [ ]:
# def mehrten_hp(upw_array, hp_value):
#     temp = np.copy(upw_array)
#     temp[-1,:,:] = hp_value
#     return(temp)
# # convert hydr. cond. from m/s to m/d
# hk = mehrten_hp(hk, 1e-5*86400)
# vka = mehrten_hp(vka, 1e-7*86400)
# sy = mehrten_hp(sy, 0.10)
# ss = mehrten_hp(ss, 1e-4)

# def laguna_hp(upw_array, hp_value):
#     temp = np.copy(upw_array)
#     temp[-2,:,:] = hp_value
#     return(temp)
# # convert hydr. cond. from m/s to m/d
# hk = laguna_hp(hk, 1e-4*86400)
# vka = laguna_hp(vka, 1e-6*86400)
# sy = laguna_hp(sy, 0.15)
# ss = laguna_hp(ss, 1e-5)

In [ ]:
# layvka 0 means vka is vert K, non zero means its the anisotropy ratio between horiz and vert
layvka = 0

# LAYTYP MUST BE GREATER THAN ZERO WHEN IUZFOPT IS 2
laytyp = 1 # 0 is confined, >0 convertible, <0 convertible unless the THICKSTRT option is in effect
# Laywet must be 0 if laytyp is confined laywet = [1,1,1,1,1]
#ipakcb = 53 means cell-by-cell budget is saved because it is non zero (default is 53)

# until upscaling is begun then vertical and horiz K are the same for TPROGS
# upw = flopy.modflow.ModflowUpw(model = m, hk =hk, layvka = layvka, vka = hk, sy=sy, ss=ss,
#                                laytyp=laytyp, ipakcb=53)

lpf = flopy.modflow.ModflowLpf(model = m, hk =hk, layvka = layvka, vka = hk, sy=sy, ss=ss,
                               laytyp=laytyp, ipakcb=53)

## GHB NW, SE set up

In [ ]:
# scaling_factors = pd.read_csv(m.model_ws+'/GHB_UZF_WEL_scaling.csv', delimiter = ',')


In [ ]:
# join top and botm for easier array referencing for elevations
top_botm = np.zeros((m.dis.nlay+1,m.dis.nrow,m.dis.ncol))
top_botm[0,:,:] = m.dis.top.array
top_botm[1:,:,:] = m.dis.botm.array


In [ ]:
distance=5000
t0 = time.time()

ghbnw_spd = pd.DataFrame(np.zeros((np.sum(nlay-nw_lay),5)))
ghbnw_spd.columns = ['k','i','j','bhead','cond']

# get all of the j,k indices to reduce math done in the for loop
yz = np.zeros((np.sum(nlay-nw_lay),2)).astype(int)
n=0
for j in np.arange(0,ncol):
    for k in np.arange(nw_lay[j], nlay):
        yz[n,0] = j
        yz[n,1] = k
        n+=1
        
condnw = lpf.hk.array[yz[:,1],0,yz[:,0]]*(top_botm[yz[:,1],0,yz[:,0]]-top_botm[yz[:,1]+1,0,yz[:,0]])*delr/distance
ghbnw_spd.cond = condnw
ghbnw_spd.bhead = avg_nw[yz[:,0]]
ghbnw_spd.k = yz[:,1]
ghbnw_spd.j = yz[:,0]
ghbnw_spd.i = 0

ghbse_spd = pd.DataFrame(np.zeros((np.sum(nlay-se_lay),5)))
ghbse_spd.columns = ['k','i','j','bhead','cond']

# get all of the j,k indices to reduce math done in the for loop
yz = np.zeros((np.sum(nlay-se_lay),2)).astype(int)
n=0
for j in np.arange(0,ncol):
    for k in np.arange(se_lay[j], nlay):
        yz[n,0] = j
        yz[n,1] = k
        n+=1
condse = lpf.hk.array[yz[:,1],int(nrow-1),yz[:,0]]*(top_botm[yz[:,1],-1,yz[:,0]]-top_botm[yz[:,1]+1,-1,yz[:,0]])*delr/distance
ghbse_spd.cond = condse
ghbse_spd.bhead = avg_se[yz[:,0]]
ghbse_spd.k = yz[:,1]
ghbse_spd.j = yz[:,0]
ghbse_spd.i = 0
        
resample_time = time.time() - t0
print("GHB time: {:.3f} sec".format(time.time() - t0))

## Southwest GHB boundary (specified head for outflow to the Delta)

How much would the expected head gradient be near the delta, how fast would head decrease with depth.
Perhaps it would only go down a few meters for every layer

In [ ]:
distance=10000

# Fine sand	2×10-7 to 2×10-4 m/s
# Silt, loess	1×10-9 to 2×10-5 m/s
# delta soils have some sand mixed in
delta_hk = 1e-5*86400

ghbdn_spd = pd.DataFrame(np.zeros(((nlay*nrow),5)))
ghbdn_spd.columns = ['k','i','j','bhead','cond']

# get all of the j,k indices to reduce math done in the for loop
xz = np.zeros((nlay*nrow,2)).astype(int)
n=0
for i in np.arange(0,nrow):
    for k in np.arange(0, nlay):
        xz[n,0] = i
        xz[n,1] = k
        n+=1
cond = delta_hk*(top_botm[xz[:,1],:,0]-top_botm[xz[:,1]+1,:,0])*delr/distance
ghbdn_spd.cond = cond
ghbdn_spd.bhead = 0
ghbdn_spd.k = xz[:,1]
ghbdn_spd.j = 0
ghbdn_spd.i = xz[:,0]

In [ ]:
# (botm[0:nlay_ghb,:,0]<0).shape
# botm[ghbdn_spd[:,:3].astype(int)].shape

In [ ]:
# lay, row, col for delta ghb
zxy = ghbdn_spd.values[:,:3].astype(int)
# (botm[zxy[:,0],zxy[:,1],zxy[:,2]]<0).shape
# drop any delta ghb cells where cell bottom is below sea level
ghbdn_spd =  ghbdn_spd.values[botm[zxy[:,0],zxy[:,1],zxy[:,2]]<0]



In [ ]:
# ghb_spd = np.vstack((ghbdn_spd, ghbup_spd))
ghb_spd = np.vstack((ghbdn_spd, ghbse_spd.values, ghbnw_spd.values))
# ghb_spd = np.vstack((ghbdn_spd, ghbup_spd, ghbse_spd, ghbnw_spd))


In [ ]:
# lay, row, col, head, cond
k = ghb_spd[:,0].astype(int)
i = ghb_spd[:,1].astype(int)
j = ghb_spd[:,2].astype(int)

# find where the ghb overlaps with the no flow cells
active_bc =bas.ibound.array[k,i,j]
# ghb_spd = ghb_spd[active_bc.astype(bool)]

In [ ]:
ghb_spd = ghb_spd[:][active_bc.astype(bool)]

In [ ]:

# GHB for east and west model boundaries
ghb.stress_period_data =  {0: ghb_spd}
# GHB for only Delta, west side of model
# ghb.stress_period_data =  {0: ghbdn_spd}

In [ ]:
ghb.check()

## CHD Package Time variant head

In [ ]:
chd = flopy.modflow.ModflowChd(model=m)

In [ ]:
# Determine which layer the specified head cell is in
# since the if statement only checks whether the first layer is greater than the streambed elevation, 
# otherwise it would be less than and zero (most should be in layer 0)
chd_lay = np.zeros(nrow)

head = dem_data[:,ncol-1]
headmin = np.min(head)
ch_weight = 0.9
chd_vals = head*(1-ch_weight)+headmin*ch_weight



for k in np.arange(0,nlay-1):
    # pull out elevation of layer bottom
    lay_elev = botm[k, :, ncol-1]
    for i in np.arange(0,nrow):
        # want to compare if streambed is lower than the layer bottom
        # 1 will be subtracted from each z value to make sure it is lower than the model top in the upper reaches
        if lay_elev[i] > chd_vals[i]:
            chd_lay[i] = k+2



In [ ]:
# layer (int), row (int), column (int), shead (float), ehead (float) shead is the head at the
# start of the stress period, and ehead is the head at the end of the stress period
# nlay_ghb = 1

# constant head boundary for mountain front recharge
# assume that near the mountains the head should be at the surface becaues the aquifer is thin

# new specified head boundary will be linear at the uppermost column to reduce nonlinearity
# as the no flow cells will be removed and replaced with low hydraulic conductivity cells

# chd_spd = np.zeros((len(chd_locs[0]),5))
chd_spd = np.zeros((int(np.sum((nlay-chd_lay))),5))

# # head for mountain front recharge
shead = chd_vals
ehead = chd_vals
p=0
for i in np.arange(0,nrow):
    for k in np.arange(0,nlay-chd_lay[i]):
        chd_spd[p] = [int(chd_lay[i]+k), i, ncol-1, shead[i], ehead[i]]
        p+=1
print('Number of CHD cells for upland bound', p)

# p = 0
# # head for mountain front recharge
# shead = head
# ehead = head

# for i, j in zip(chd_locs[0], chd_locs[1]):
#     chd_spd[p] = [0, i, j, shead[p], ehead[p]]
#     p+=1
# print('Number of CHD cells for upland bound', p)


In [ ]:
chd.stress_period_data =  {0: chd_spd}

In [ ]:
chd.check()

In [ ]:
# chd.write_file()

The code that originally imported the state soil map data was moved up above to be readily applicable to the LAK package and SFR package. gpd_mb and grid_uzf are created above

## RCH Package
1. Load in already interpolated ET and rainfall data
2. Determine where there is agricultural land to add irrigation multiplied by an efficiency factor to the rainfall array
3. Difference the rainfall and ETc arrays to dtermine how much water will recharge the aquifer
*May need to make ET and Rainfall arrays available offline on your computer*

In [ ]:
# ETc = np.zeros((nper-1,nrow,ncol))
# rain = np.zeros((nper-1,nrow,ncol))

# ETc_count = 0
# for y in np.arange(pd.to_datetime(strt_date).year, pd.to_datetime(end_date).year+1):
#     # set start and end date for range for the year to be iterated over
#     yr_strt = pd.to_datetime(str(y)+'-01-01')
#     yr_end = pd.to_datetime(str(y)+'-12-31')
#     if yr_strt < pd.to_datetime(strt_date):
#         yr_strt = pd.to_datetime(strt_date)
#     if yr_end > pd.to_datetime(end_date):
#         yr_end = pd.to_datetime(end_date)
        
#     # read in text file of all of the ETc data for each year in array format    
#     ET_year = np.loadtxt(gwfm_dir+'/UZF_data/ETa_all_txt_arrays/ETa_array_'+str(y)+'.tsv', delimiter = '\t')
#     rain_year = np.loadtxt(gwfm_dir+'/UZF_data/Rain_all_txt_arrays/Rain_array_'+str(y)+'.tsv', delimiter = '\t')

#     # get the length of the date range needed for that year
#     yearlen = len(pd.date_range(yr_strt, yr_end))
#     # correct the shape of the text file from 2D to 3D
#     revertETc = np.reshape(ET_year, (yearlen, nrow, ncol))
#     revertrain = np.reshape(rain_year, (yearlen, nrow, ncol))
#     # filter the 3D array based on the desired date range
#     filtered_date_ETc = revertETc[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
#     filtered_date_rain = revertrain[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
#     # add the data to the ETc array for the whole model time period
#     ETc[ETc_count:ETc_count+yearlen,:,:] = filtered_date_ETc
#     rain[ETc_count:ETc_count+yearlen,:,:] = filtered_date_rain
#     ETc_count += ETc_count+yearlen

In [ ]:

# logical array where 1 represents ag land and 0 represents non ag land as listed above
ag_land_arr_regional = np.loadtxt(gwfm_dir+'/UZF_data/array_of_agriculture_land.tsv')
# get for inset model
ag_land_arr = np.zeros((nrow,ncol))
ag_land_arr[x_in,y_in] = ag_land_arr_regional[x,y]

# create a bool array of the oppostite of ag land
non_ag_land_arr = ~ag_land_arr.astype(bool)


In [ ]:
# wherever there is ag land and not enough rain to meet the ETc then it will be met by applying 1.05*ETc
# to the .FINF array
# start by setting the rain of non_ag_land to np.nan to be ignored in calculations
# ag_rch = np.copy(rain)
# ag_rch[:,non_ag_land_arr] = np.nan
# # whereever the rain is less than ETc for ag land set the recharge equal to the etc times 0.05 for irrigation inefficiency
# # could calibrate this irrigation efficiency later
# ag_rch[ag_rch<ETc] = ETc[ag_rch<ETc]*0.05
# # return np.nan values to zeros
# ag_rch[np.isnan(ag_rch)] = 0

In [ ]:
# final calculation for RCH package

# net_inf = rain + ag_rch - ETc
# # assuming there is irrigation to make up lack of rainfall
# # net_inf should never be less than zero
# np.mean(net_inf, axis = (1,2))

# net_inf = np.where(net_inf<0, 0, net_inf) 
# convert 20 inches /year to m/day
rch_rate = (20*0.0254)/365
net_inf = np.ones((nrow,ncol))*rch_rate

In [ ]:
# add an empty array for the first blank stress period
# net_inf = np.vstack((np.zeros((1,nrow,ncol)),net_inf))

In [ ]:
# have transient recharge start after the 1st spd
rech_spd = { 0:net_inf}

In [ ]:
# nrchop = 3, to highest active cell
rch =flopy.modflow.ModflowRch(model = m, nrchop=3, rech = rech_spd)

In [ ]:
# rch.write_file()

## Well Package


In [ ]:
wells = pd.read_csv(gwfm_dir+'/WEL_data/all_wells_type.csv')
wells_grid = gpd.GeoDataFrame(wells, geometry = gpd.points_from_xy(wells.easting,wells.northing), crs = 'epsg:32610')
# prepare for inset grid
wells_grid = wells_grid.rename(columns = {'row':'row_regional','column':'column_regional'})
wells_grid= wells_grid.set_index(['row_regional','column_regional'])
wells_grid = wells_grid.join(grid_ref.set_index(['row_regional','column_regional']), how='inner',rsuffix = 'grid' )
wells_grid = wells_grid.reset_index()

In [ ]:
wells_grid_ag = wells_grid[:]
wells_grid_ag = wells_grid.loc[wells_grid.Simple_type == 'irrigation'].dissolve('node', aggfunc = 'mean')
wells_grid_ag.geometry = wells_grid.loc[wells_grid.Simple_type == 'irrigation'].dissolve('node', aggfunc = 'first').geometry

In [ ]:
wells_grid['depth_m'] = wells_grid.TotalCompletedDepth*0.3048


In [ ]:
wells_grid['flux'] = 0
wells_grid['layer'] = 0

In [ ]:
# rough ETc calc
et_rate = 1.4/365
ETc = np.ones((nper, nrow,ncol))*et_rate

In [ ]:
num_ag = (wells_grid.Simple_type == 'irrigation').sum()

ET_ag = ETc*ag_land_arr

# ET_ag_monthly_sum = ET_ag.sum(axis=(1,2))
# # calculate ag well flux by average ET
# aggregated_irrig_flux = ET_ag_monthly_sum/num_ag


In [ ]:
# assume each public supply well serves 5-10,000 people each needing 50 gpd, then need to convert to ft^3
public_flux = (5000*50/7.48)*(0.3048**3)
# public_flux = 1500*(1/7.48)*(0.3048**3)*60*24

# averge pumping rate of domestic wells is 10-100 gpm, typically on the lower end and 
# should end up being around 50 gal/person a day and average 3 people is 150 gal/day
# 10 gpm * (1 ft^3/ 7.48 gal) (ft^3/m^3)*60 min hour* 3 hours in a day * 30 days

dom_flux = 30*(1/7.48)*(0.3048**3)*60*24
print('Irrig flux:', '%.3e' % (ET_ag.mean()*270*270), 'Public flux:', '%.3e' %public_flux,'Domestic flux:', '%.3e' %dom_flux)

In [ ]:
# define pumping rate based on well use, average pumping rate in m^3/day
# wells_grid.loc[wells_grid.Simple_type == 'irrigation', 'flux'] = -irrig_flux
wells_grid.loc[wells_grid.Simple_type == 'domestic', 'flux'] = -dom_flux
wells_grid.loc[wells_grid.Simple_type == 'public', 'flux'] = -public_flux

In [ ]:
if wells_grid.row.min()==1:
    wells_grid.row = (wells_grid.row-1).astype(int)
    wells_grid.column = (wells_grid.column -1).astype(int)

In [ ]:
mean_ag_well_depth = wells_grid.loc[wells_grid.Simple_type == 'irrigation'].TotalCompletedDepth.mean()*0.3048
mean_ag_well_depth

In [ ]:

for k in np.arange(0,nlay-1):
    # pull out elevation of layer bottom
    lay_elev = botm[k, :, :]
    for i in np.arange(0,len(wells_grid)):
        # want to compare if streambed is lower than the layer bottom
        # 1 will be subtracted from each z value to make sure it is lower than the model top in the upper reaches
        if lay_elev[wells_grid.row.values[i],wells_grid.column.values[i]] > dem_data[wells_grid.row.values[i],wells_grid.column.values[i]]-wells_grid.depth_m.iloc[i]:
            wells_grid.layer.iloc[i] = k     

In [ ]:
wells_grid.layer

In [ ]:
print('Cells with wells: ', wells_grid.dissolve(by='node',aggfunc='first').shape[0], 'total wells: ', wells_grid.shape[0])
print('Wells with TRS accuracy: ', (wells_grid.MethodofDeterminationLL == 'Derived from TRS').sum())

wells_grid_notrs = wells_grid.loc[wells_grid.MethodofDeterminationLL != 'Derived from TRS']

In [ ]:
# wells_grid_notrs['count_per_cell'] = 1
# # all_wells_grid['MFCell'] = all_wells_grid.Township+all_wells_grid.Range+all_wells_grid.k.astype(str)
# wells_grid_sum = wells_grid_notrs.dissolve(by = 'node', aggfunc='sum')
# wells_grid_sum.plot('count_per_cell',legend=True)
# print(wells_grid_sum.count_per_cell.median(), wells_grid_sum.count_per_cell.mean(), wells_grid_sum.count_per_cell.max())


In [ ]:
# save domestic, public wells for their pumping doesn't depend on ET
wells_grid_no_ag = wells_grid.loc[wells_grid.Simple_type != 'irrigation']
# filter out data for wel package
spd_noag = wells_grid_no_ag.loc[:,['layer','row','column', 'flux']].values


#### Using ETc in all cells for pumping


In [ ]:
# layer for ETc ag well pumping
ETc_lay = 1
# create empty dictionary to fill with stress period data
wel_ETc_dict = {}
# end date is not included as a stress period, starting at 1st TR spd (2)
for t in np.arange(1,nper):
    wel_i, wel_j = np.where(ET_ag[t-1,:,:]>0)
    wel_ETc = -ET_ag[t-1,wel_i,wel_j]*delr*delr
    # ['layer','row','column', 'flux'] are necessary for WEL package
    spd_ag = np.stack((ETc_lay*np.ones(len(wel_i)), wel_i, wel_j,wel_ETc),axis=1)
    spd_all = np.vstack((spd_ag,spd_noag)) # join pumping from ag with point pumping from domstic/supply wells that are constant
    wel_ETc_dict[t] = spd_all

In [ ]:
ET_ag_ss = ET_ag.mean(axis=(0))
wel_i, wel_j = np.where(ET_ag_ss[:,:]>0)
wel_ETc = -ET_ag_ss[wel_i,wel_j]*delr*delr
# ['layer','row','column', 'flux'] are necessary for WEL package
# add average ag well pumping as steady state period
wel_ETc_dict[0] = np.stack((ETc_lay*np.ones(len(wel_i)), wel_i, wel_j,wel_ETc),axis=1)

In [ ]:
# Create well flopy object
wel = flopy.modflow.ModflowWel(m, stress_period_data=wel_ETc_dict,ipakcb=55)

## Output Control

In [ ]:
# Output control
# default unit number for heads is 51, cell by cell is 53 and drawdown is 52
# (0,0) is (stress period, time step)

# For later model runs when all the data is needed to be saved
spd = { (j,0): ['save head', 'save budget'] for j in np.arange(0,nper,1)}

# get the first of each month to print the budget
month_intervals = (pd.date_range(strt_date,end_date, freq="MS")-pd.to_datetime(strt_date)).days

for j in month_intervals:
    spd[j,0] = ['save head', 'save budget','print budget']
    
oc = flopy.modflow.ModflowOc(model = m, stress_period_data = spd, compact = True)

## Newton Solver

In [ ]:
# pcg = flopy.modflow.ModflowPcg(model = m)
# nwt = flopy.modflow.ModflowNwt(model= m)
# thickfact: portion of cell thickness used for smoothly adjusting storage and conductance coefficients to zero (default is 1e-5)
# linmeth (linear method): 1 for GMRES and 2 for XMD (1 is default)
# iprnwt: flag whether additional info about solver convergence will be printed to the main listing file (default is 0)
# ibotav: flag whether corretion will be made to gw head relative to cell-bottom if surrounded by dry cells.
# 1 = corrections and  0 = no correction (default is 0)
# options: specify comlexity of solver. SIMPLE : default solver for linear models, MODERATE for moderately nonlinear models,
# COMPLEX for highly nonlinear models (default is COMPLEX)
# Continue: if model fails to converge during a time step it will continue to solve the next time step (default is False) 
# epsrn (XMD) is the drop tolerance for preconditioning (default is 1E-4)
# hclosexmd (XMD) head closure criteria for inner (linear) iterations (default 1e-4)

# nwt = flopy.modflow.ModflowNwt(model = m, headtol=0.01, fluxtol=500, maxiterout=200, thickfact=1e-05, 
#                                linmeth=1, iprnwt=1, ibotav=0, options='COMPLEX', Continue=False,
#                                maxbackiter=50, backtol=1.1, maxitinner=50, ilumethod=2, 
#                                levfill=5, stoptol=1e-10, msdr=15, iacl=2, norder=1, level=5, north=7, 
#                                iredsys=0, rrctols=0.0, idroptol=1, epsrn=0.0001, hclosexmd=0.0001, 
#                                mxiterxmd=50, extension='nwt', unitnumber=None, filenames=None)



# GMG is more successful than pcg which is fine for steady state model
# mxiter, max outer, iiter = max inner, hclose = head change criterion for convergence, 
# rclose = residual criterion for convergence

# gmg = flopy.modflow.ModflowGmg(model = m, mxiter=50, iiter=30, hclose = 1e-5, rclose = 1e-5)

In [ ]:
max_outer = 200
max_inner = 100
# iter_mo max outer iters, ter_mi = max inner iters, close_r residual criterion for stopping iteration
# close_h is alternate criterion for nonlinear problem, and is head closure which should be smaller than residual closer
# ipunit =0 means no info on solver, ipunit=1 means output about solver issues is written
# if iter_mo >1 then closer_r is used not close_h and closer_r is compared to 
# the square root of the inner product of the residuals (the residual norm)
# adamp =0 is std damping, adamp=1 is adaptive damping that further decreases or increases damping based on picard
# iteration sucess
#adamp is 0.7 to resolve issues with heads oscillating near solution +1 m
# damp_lb = lower bound, rate_d is rate of increase of damping based picard iteration success
pcgn = flopy.modflow.ModflowPcgn(m, iter_mo = max_outer, iter_mi=max_inner, close_r=1e-01, close_h=1e-02, ipunit=28)
#                                adamp=1, damp=0.7, damp_lb=0.1, rate_d=0.01)

pcgn.write_file()

In [ ]:
m.get_package_list()
# m.remove_package('DATA')
# m.remove_package('RIV')
# m.remove_package('WEL')
# m.remove_package('RCH')


In [ ]:

m.check()
# lak.check()
# upw.check()

## Write the input files

In [ ]:
# Writing the MODFLOW data files
m.write_input()


# Run the model

In [ ]:
# success, buff = m.run_model()
# if not success:
#     raise Exception("MODFLOW did not terminate normally.")

In [ ]:
# hdobj = flopy.utils.HeadFile(model_ws+'/MF.hds')
# head = hdobj.get_data(kstpkper = (0,0))

In [ ]:
# layer 9 is 4.5m
# 50m-4.5 45.5m and 45m is the bottom
# 45.03775 which is below
# chd_vals[49]

In [ ]:
chd.check()

In [ ]:
# m.riv.stress_period_data[0]

In [ ]:
wel.stress_period_data[0].flux.sum(), rch.rech.array[0,0,:,:].sum()*270*270

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

ax.plot(chd_vals, label = 'chd')
for n in np.arange(95,105):
    ax.plot(strt[0,:,n],label='strt heads '+str(n))

plt.legend(loc=(1,0.2))

In [ ]:
def plot_lines(lines):
    fig, ax = plt.subplots()
    for ln in lines:
        plt.plot([ln[0][0], ln[1][0]], [ln[0][1], ln[1][1]], linewidth = 0.1)

In [ ]:
# Plot the basic model grid and sensors
plt.rcParams["figure.figsize"] = (20,10)
#fig = plot_lines(mg.grid_lines)
plt.scatter(x = mg.extent[0:2], y = mg.extent[2:])
# plot_lines(mg.grid_lines)
plt.scatter(east, north, c='r')

 insufficient virtual memory means there is way too much data for the fortran arrays too hold, more than the 16 gb I have available